# Clustering Consulting Project - Solutions

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [1]:
# Initiate Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("hack_find").getOrCreate()

## Load the data

In [2]:
dataset = spark.read.csv('resources/hack_data.csv', inferSchema=True, header=True)

dataset.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [4]:
dataset.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58|
|                    2.0|           228.08|              1|             2.48|            8.0|             Bolivia|            70.8|
|                   20.0|            408.5|              0|             3.57

In [5]:
dataset.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

## Data Transformation

In [6]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used', 
                                       'Servers_Corrupted', 'Pages_Corrupted', 'WPM_Typing_Speed'], 
                            outputCol='features')

final_data = assembler.transform(dataset)

final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)



## Scale the data for clustering

In [7]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures', withStd=True, withMean=False)

final_data = scaler.fit(final_data).transform(final_data)

In [8]:
final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaledFeatures: vector (nullable = true)



## Perform KMeans clustering

In [9]:
# We will use both k=3 and k=2
from pyspark.ml.clustering import KMeans

kmeans3 = KMeans(featuresCol='scaledFeatures', k=3)
kmeans2 = KMeans(featuresCol='scaledFeatures', k=2)

model_k3 = kmeans3.fit(final_data)
model_k2 = kmeans2.fit(final_data)

print(f"With K=2 Within Set Sum of Squared Errors = {model_k2.computeCost(final_data)}")
print(f"With K=3, Within Set Sum of Squared Errors = {model_k3.computeCost(final_data)}")

With K=2 Within Set Sum of Squared Errors = 601.7707512676716
With K=3, Within Set Sum of Squared Errors = 434.1492898715845


Now, we know that as the value of k increases, Within Set Sum of Squared Errors also decreases....

We would like to use a range of K and compute the cost and find out the optimal K

In [10]:
for k in range(2, 9):
    kmeans = KMeans(featuresCol='scaledFeatures', k=k)
    model = kmeans.fit(final_data)
    wssse = model.computeCost(final_data)
    print(f"With K as {k}, within Set Sum of Squared Errors is : {wssse}")
    print('--'*30)

With K as 2, within Set Sum of Squared Errors is : 601.7707512676716
------------------------------------------------------------
With K as 3, within Set Sum of Squared Errors is : 434.1492898715845
------------------------------------------------------------
With K as 4, within Set Sum of Squared Errors is : 267.1336116887891
------------------------------------------------------------
With K as 5, within Set Sum of Squared Errors is : 253.2614967829523
------------------------------------------------------------
With K as 6, within Set Sum of Squared Errors is : 395.95122756923143
------------------------------------------------------------
With K as 7, within Set Sum of Squared Errors is : 206.60565268596142
------------------------------------------------------------
With K as 8, within Set Sum of Squared Errors is : 200.86949274752277
------------------------------------------------------------


** Nothing definitive can be said with the above, but wait! The last key fact that the engineer mentioned was that the attacks should be evenly numbered between the hackers! Let's check with the transform and prediction columns that result form this! Congratulations if you made this connection, it was quite tricky given what we've covered!**

In [11]:
# We will check the count by cluster sizes to see whichones are evenly numbered

model_k3.transform(final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   84|
|         2|   83|
|         0|  167|
+----------+-----+



In [12]:
# We will check with 2 clusters

model_k2.transform(final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



### Bingo! It was 2 hackers, in fact, our clustering algorithm created two equally sized clusters with K=2, no way that is a coincidence!